In [1]:
import findspark
findspark.init()
import pyspark
from random import random
from pyspark import SparkContext
import numpy as np
from math import sqrt

import matplotlib
import matplotlib.pyplot
import matplotlib.pyplot as plt
%matplotlib inline 

 

In [2]:
config = pyspark.SparkConf().set('spark.executor.memory', '512M').set('spark.cores.max', '2')\
            .set('spark.port.maxRetries','200') 
sc = pyspark.SparkContext(appName="Project #2 - Santander", master='spark://polyp1:7077',conf = config )

# The Problem
We are going to participate in a Kaggle competition here
https://www.kaggle.com/c/santander-product-recommendation

I have already downloaded the data to our cluster + I have also created a small and medium dataset for you to start.
<br>
This small dataset contains information about randomly chosen 1,000 customers
<br>
This medium dataset contains information about randomly chosen 10,000 customers
<br>
<br>
Each Team has to register to the competition. Read how to submit your solution to the website and also try to do it. We will like to see that you tried to submit your solution and also part of the project is to tell us how you did. You have to report the best score you obtained + also attach your "subission" file.
## Note
We do not expect you to be the best, this would take a lot of time, we just want you to learn a lot by working on this project

In [3]:
# you will need this data once you get familiar with the problem, 
# it is better to start with the small subset defined below
# trainData = sc.textFile('/scratch/ISE495/2017_project_02/train_ver2.csv')
# testData = sc.textFile('/scratch/ISE495/2017_project_02/test_ver2.csv')
# sampleSubmission = sc.textFile('/scratch/ISE495/2017_project_02/sample_submission.csv')

#trainData = sc.textFile('/scratch/ISE495/2017_project_02/train_ver2_medium.csv')
trainData = sc.textFile('/scratch/ISE495/2017_project_02/train_ver2_small.csv')

In [4]:
print trainData.take(5)

[u'"fecha_dato","ncodpers","ind_empleado","pais_residencia","sexo","age","fecha_alta","ind_nuevo","antiguedad","indrel","ult_fec_cli_1t","indrel_1mes","tiprel_1mes","indresi","indext","conyuemp","canal_entrada","indfall","tipodom","cod_prov","nomprov","ind_actividad_cliente","renta","segmento","ind_ahor_fin_ult1","ind_aval_fin_ult1","ind_cco_fin_ult1","ind_cder_fin_ult1","ind_cno_fin_ult1","ind_ctju_fin_ult1","ind_ctma_fin_ult1","ind_ctop_fin_ult1","ind_ctpp_fin_ult1","ind_deco_fin_ult1","ind_deme_fin_ult1","ind_dela_fin_ult1","ind_ecue_fin_ult1","ind_fond_fin_ult1","ind_hip_fin_ult1","ind_plan_fin_ult1","ind_pres_fin_ult1","ind_reca_fin_ult1","ind_tjcr_fin_ult1","ind_valo_fin_ult1","ind_viv_fin_ult1","ind_nomina_ult1","ind_nom_pens_ult1","ind_recibo_ult1"', u'2015-01-28,1050781,N,ES,H, 23,2012-08-10, 0,     35, 1,,1.0,I,S,N,,KHE,N, 1,16,"CUENCA", 0,69094.08,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0, 0,0', u'2015-01-28,1052003,N,ES,H, 22,2012-08-16, 0,     35, 1,,

In [4]:
filtedTitle =  trainData.filter(lambda l:l.split(',')[0].find('fecha') == -1).count()
print filtedTitle

14348


In [5]:
filtedTitle =  trainData.filter(lambda l:l.split(',')[0].find('fecha') == -1)
print filtedTitle.count()
filtedNA =  filtedTitle.filter(lambda l:l.find('NA') == -1)
print filtedNA.count()

14348
12489


In [34]:
print filtedTitle.filter(lambda l:l.find('NA') == -1).count()

124129


In [45]:
def NAtozero(e):
    lis = e.split(',')
    if lis[24] != '0' or lis[24] != '1':
        del lis[24]   
    for ele in lis[24:48]:
        if ele.find('NA') != -1:
            ele = '0'
    return [lis[1],lis[24:48]]
def rating(e):
    return [[e[0],i,e[1][i]] for i in range(len(e[1]))]
NA__0 = filtedNA.map(NAtozero)
rates =  NA__0.flatMap(rating)
      

In [46]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
ratings = rates.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
print ratings.take(5)

[Rating(user=1050781, product=0, rating=0.0), Rating(user=1050781, product=1, rating=1.0), Rating(user=1050781, product=2, rating=0.0), Rating(user=1050781, product=3, rating=0.0), Rating(user=1050781, product=4, rating=0.0)]


[u'2015-01-28', u'1050781', u'2015-01-28', u'1052003', u'2015-01-28', u'1051676', u'2015-01-28', u'1048810', u'2015-01-28', u'1050023']


AttributeError: 'list' object has no attribute 'flatMap'

[1]


0
2
4
6
8


0
1
4


<generator object gen at 0x7f0d01a0fdc0>
0
1
2


In [ ]:
sc.stop()